# MMR (Maximal Marginal Relevance)

In [7]:
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [2]:
documents = [
    Document(page_content="LangChain helps developers build LLM applications easily."),
    Document(page_content="Chroma is a vector database optimized for LLM-based search."),
    Document(page_content="Embeddings convert text into high-dimensional vectors."),
    Document(page_content="OpenAI provides powerful embedding models."),
]

In [9]:
embeddings = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(documents, embeddings)

In [13]:
# Enable MMR in the retriever

retriever = vectorstore.as_retriever(
    search_type="mmr",      # <-- This enables MMR
    search_kwargs={"k": 3, "lambda_mult": 0.5},
)


In [14]:
query = "What is LangChain?"

results = retriever.invoke(query)

In [15]:
for i, doc in enumerate(results):
    print(f"Result {i+1}:")
    print(doc.page_content)
    print()

Result 1:
LangChain helps developers build LLM applications easily.

Result 2:
Embeddings convert text into high-dimensional vectors.

Result 3:
Chroma is a vector database optimized for LLM-based search.



## MultiQuery Retriever

In [1]:
from langchain.retrievers import MultiQueryRetriever
from langchain_community.vectorstores import FAISS
from langchain_huggingface import HuggingFaceEmbeddings
from langchain_core.documents import Document

In [2]:
# Relevant health & wellness documents
all_docs = [
    Document(page_content="Regular walking boosts heart health and can reduce symptoms of depression.", metadata={"source": "H1"}),
    Document(page_content="Consuming leafy greens and fruits helps detox the body and improve longevity.", metadata={"source": "H2"}),
    Document(page_content="Deep sleep is crucial for cellular repair and emotional regulation.", metadata={"source": "H3"}),
    Document(page_content="Mindfulness and controlled breathing lower cortisol and improve mental clarity.", metadata={"source": "H4"}),
    Document(page_content="Drinking sufficient water throughout the day helps maintain metabolism and energy.", metadata={"source": "H5"}),
    Document(page_content="The solar energy system in modern homes helps balance electricity demand.", metadata={"source": "I1"}),
    Document(page_content="Python balances readability with power, making it a popular system design language.", metadata={"source": "I2"}),
    Document(page_content="Photosynthesis enables plants to produce energy by converting sunlight.", metadata={"source": "I3"}),
    Document(page_content="The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.", metadata={"source": "I4"}),
    Document(page_content="Black holes bend spacetime and store immense gravitational energy.", metadata={"source": "I5"}),
]

In [3]:
embedding_model = HuggingFaceEmbeddings()

vectorstore = FAISS.from_documents(documents=all_docs, embedding=embedding_model)

e:\HashimEdisk\PracticeAndProjects\LangChain\langchain_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Let's try the difference

In [4]:
# create similarity retriever
similarity_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 5}
)

In [5]:
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

True

In [20]:
# creating the MQR
mq_retriever = MultiQueryRetriever.from_llm(
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3}),
    llm = ChatGroq(model_name="llama3-8b-8192"),
)

In [21]:
query = "How to improve energy levels and maintain balance?"

In [22]:
## retrieve results
similarity_result = similarity_retriever.invoke(query)
multiquery_result = mq_retriever.invoke(query)

In [23]:
for i, doc in enumerate(similarity_result):
    print(f"Result {i+1}:")
    print(doc.page_content)

print("*"*150)

for i, doc in enumerate(multiquery_result):
    print(f"Result {i+1}:")
    print(doc.page_content)

Result 1:
Drinking sufficient water throughout the day helps maintain metabolism and energy.
Result 2:
Regular walking boosts heart health and can reduce symptoms of depression.
Result 3:
Mindfulness and controlled breathing lower cortisol and improve mental clarity.
Result 4:
Consuming leafy greens and fruits helps detox the body and improve longevity.
Result 5:
The solar energy system in modern homes helps balance electricity demand.
******************************************************************************************************************************************************
Result 1:
Black holes bend spacetime and store immense gravitational energy.
Result 2:
Photosynthesis enables plants to produce energy by converting sunlight.
Result 3:
The 2022 FIFA World Cup was held in Qatar and drew global energy and excitement.
Result 4:
Regular walking boosts heart health and can reduce symptoms of depression.
Result 5:
Drinking sufficient water throughout the day helps maintain meta

## Contextual Compression Retriever

In [24]:
from langchain.retrievers.contextual_compression import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import LLMChainExtractor

In [25]:
# Recreate the document objects from the previous data
docs = [
    Document(page_content=(
        """The Grand Canyon is one of the most visited natural wonders in the world.
        Photosynthesis is the process by which green plants convert sunlight into energy.
        Millions of tourists travel to see it every year. The rocks date back millions of years."""
    ), metadata={"source": "Doc1"}),

    Document(page_content=(
        """In medieval Europe, castles were built primarily for defense.
        The chlorophyll in plant cells captures sunlight during photosynthesis.
        Knights wore armor made of metal. Siege weapons were often used to breach castle walls."""
    ), metadata={"source": "Doc2"}),

    Document(page_content=(
        """Basketball was invented by Dr. James Naismith in the late 19th century.
        It was originally played with a soccer ball and peach baskets. NBA is now a global league."""
    ), metadata={"source": "Doc3"}),

    Document(page_content=(
        """The history of cinema began in the late 1800s. Silent films were the earliest form.
        Thomas Edison was among the pioneers. Photosynthesis does not occur in animal cells.
        Modern filmmaking involves complex CGI and sound design."""
    ), metadata={"source": "Doc4"})
]

In [26]:
embedding_model = HuggingFaceEmbeddings()
vectorstore = FAISS.from_documents(docs, embedding_model)

In [27]:
base_retriever = vectorstore.as_retriever(search_kwargs={"k": 5})
compressor = LLMChainExtractor.from_llm(ChatGroq(model_name="llama3-8b-8192"))

In [28]:
# create the contextual compression retriever
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor,
    base_retriever=base_retriever,
)

In [29]:
query = "What is photosyntheses?"
compressed_results = compression_retriever.invoke(query)

In [31]:
for i, doc in enumerate(compressed_results):
    print(f"Result {i+1}:")
    print(doc.page_content)

Result 1:
Extracted relevant parts:

Photosynthesis is the process by which green plants convert sunlight into energy.
Result 2:
The extracted relevant parts are:

The chlorophyll in plant cells captures sunlight during photosynthesis.
